In [1]:
from sklearn import preprocessing
import numpy as np
import random
import pickle

In [2]:
import math

In [3]:
#model 1 imports
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

In [4]:
import torch.utils.data as Data

In [30]:
#model 2 get inputs both_vectors
with open('new_both_vectors.dat','rb')as file: 
    both_vectors=pickle.load(file)
print(both_vectors.shape)

(151444, 72, 32)


In [31]:
#model 3
#打乱顺序 result结果设为0 onehot labels
sample_len = int(both_vectors.shape[0]/2)
results0 = np.zeros(sample_len)
results1 = np.ones(sample_len)
results = np.concatenate((results0,results1),axis = 0)
print(results.shape)

results_oppo = np.concatenate((results1,results0),axis = 0)
results_onehot = np.concatenate((results[:,np.newaxis],results_oppo[:,np.newaxis]),axis = 1)
print("made one-hot labels of shape:")
print(results_onehot.shape)

(151444,)
made one-hot labels of shape:
(151444, 2)


In [16]:
batch_size = 32
val_size_half = 1000
v1 = both_vectors[0:val_size_half]
v2 = both_vectors[151443-val_size_half:151443]
val_data = np.concatenate((v1,v2),axis = 0)
t1 = results_onehot[0:val_size_half]
t2 = results_onehot[151443-val_size_half:151443]
val_target = np.concatenate((t1,t2),axis = 0)
print("validation dataset shape:",val_data.shape)
print("validation target shape:",val_target.shape)
val_dataset_onehot = Data.TensorDataset(torch.from_numpy(val_data), torch.from_numpy(val_target))
#创建一个dataloader类的实例
val_loader_onehot = Data.DataLoader(
    # 从数据库中每次抽出batch size个样本
    dataset=val_dataset_onehot,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
)

validation dataset shape: (2000, 72, 32)
validation target shape: (2000, 2)


In [33]:
#model 4 get subset dataloader of samples
feed_size_half = 40000
start = sample_len-feed_size_half
end = feed_size_half+sample_len
#print(start,end)
batch_size = 32
small_dataset_onehot = Data.TensorDataset(torch.from_numpy(both_vectors[start:end]), torch.from_numpy(results_onehot[start:end]))
#创建一个dataloader类的实例
small_loader_onehot = Data.DataLoader(
    # 从数据库中每次抽出batch size个样本
    dataset=small_dataset_onehot,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
)

In [28]:
###################################################################################
# ensenmble bi-lstm model
sequence_length = 32
input_size = 32 #emebedding vector feature
hidden_size = 128 #hidden_size = 128 works for 10000data，acc=65
num_layers = 3
num_classes = 2
batch_size = 32
num_epochs = 100
learning_rate = 0.001
weight_decay = 0

In [11]:
class subBRNN(nn.Module):
    def _init_lstm(self, weight):
        for w in weight.chunk(4, 0):
            nn.init.xavier_uniform_(w)
        



    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(subBRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers, batch_first=True, bidirectional=True
        )
        


    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size)

        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]

        return out

In [26]:
class MyEnsemble(nn.Module):
    def __init__(self, modelA, modelB, input_len,nb_classes=2):
        super(MyEnsemble, self).__init__()
        #self.bn1 = nn.BatchNorm1d(sequence_length)
        #self.bn2 = nn.BatchNorm1d(sequence_length)
        self.ln1 = torch.nn.LayerNorm([sequence_length,input_size],elementwise_affine=False)
        
        self.ln2 = torch.nn.LayerNorm([sequence_length,input_size],elementwise_affine=False)
        
        
        self.modelA = modelA
        self.modelB = modelB
        
        self.bn3 = nn.BatchNorm1d(nb_classes)
        
        # Create new classifier
        self.classifier = nn.Linear(hidden_size*4, hidden_size)
        self.dropout1 = nn.Dropout(p = 0.3)
        self.classifier2 = nn.Linear(hidden_size, nb_classes)
        self.dropout2 = nn.Dropout(p = 0.3)
        
        nn.init.xavier_uniform_(self.classifier.weight,gain=math.sqrt(2.0))
        nn.init.xavier_uniform_(self.classifier2.weight,gain=math.sqrt(2.0))
        #linear两层会经常输出全相同
        
        
        
        
    def forward(self, xa,xb):
        #xa = self.ln1(xa)
        #xb = self.ln2(xb)
        #xa = self.bn1(xa)
        #xb = self.bn2(xb)
        x1 = self.modelA(xa)  # condition
        
        x2 = self.modelB(xb)  # raise
        
        x_con = torch.cat((x1, x2), dim=1)
        #print("x_con",x_con.shape)
        
        
        #x = self.classifier(F.leaky_relu_(x_con))
        x_c1 = self.classifier(x_con)
        #x_c1 = self.bn(x_c1)
        x_c1 = self.dropout1(x_c1)
        x_c2 = self.classifier2(x_c1)
        x_c2 = self.dropout2(x_c2)
        #return F.log_softmax(x,dim=-1)
        #x_c2 = self.bn3(x_c2)
        
        return x_c2
    
modelA = subBRNN(input_size, hidden_size, num_layers, num_classes)
modelB = subBRNN(input_size, hidden_size, num_layers, num_classes)
model = MyEnsemble(modelA, modelB,hidden_size * 4)


In [29]:
# Train double Network !!!!!!!!!!!!!!

# 定义loss和optimizer
#criterion = nn.CrossEntropyLoss()
val_losses = []
val_acces = []
losses = []
acces = []
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay= weight_decay)

for epoch in range(num_epochs):
    correct = torch.zeros(1)
    total = torch.zeros(1)
    sum_loss = torch.zeros(1)
    batches = torch.zeros(1)
    #each epoch learn different sub-dataset
    '''
    if(epoch%4==0):
        loader = subloader1
    elif(epoch%4==1):
        loader = subloader2
    elif(epoch%4==2):
        loader = subloader3
    else:
        loader = subloader4
    
    '''
    #loader = subloaders[epoch%9]
    for batch_idx, (data, targets) in enumerate(small_loader_onehot):#80000  #small_loader_onehot loader 20000 ,loader80k
        # Get data to cuda if possible
        #data = data.to(device=device).squeeze(1)
        #targets = targets.to(device=device)
        
        # forward
        data = data.float() 
        targets = targets.float() 
        
        
        
        scores = model(data[:,0:16,:],data[:,32:32+20,:])
        #print("scores",scores)
        
        loss = criterion(scores, targets)
        #loss=F.nll_loss(scores,targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
        
        
        
        if (batch_idx+1) % 512 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, batch_idx+1, 0, loss.item()))
            #print(scores)
            
        prediction = torch.argmax(scores,dim=1)
        #bce entropy
        curr_correct = (prediction == targets[:,1]).sum().float()
        
        #cross entropy 
        #curr_correct = (prediction == targets).sum().float()
        
        correct += curr_correct                
        total += len(targets)
        sum_loss += loss.item()
        batches += 1
        if(False):
            if(batch_idx<7):
                print("prediction")
                print(prediction)
        
        if(False):
           
            print("prediction",prediction)
            print("targets",targets[:,1])
            print("score",scores)
            print("curr_correct",curr_correct)
        
    #validation-------------------------------------------------------------------------------------
    model.eval()
    val_loss_sum = torch.zeros(1)
    val_batches = torch.zeros(1)
    val_correct = torch.zeros(1)
    val_total = torch.zeros(1)
    #val_acc_sum = 0.0
    for val_step,  (val_data, val_targets) in enumerate(val_loader_onehot, 1):
        with torch.no_grad():
            val_data = val_data.float() 
            val_targets = val_targets.float()
            val_scores = model(val_data[:,0:16,:],val_data[:,32:32+20,:])
            val_loss = criterion(val_scores, val_targets)
            
            val_prediction = torch.argmax(val_scores,dim=1)
            valcurr_correct = (val_prediction == val_targets[:,1]).sum().float()
        val_correct += valcurr_correct     
        val_loss_sum += val_loss.item()
        val_batches += 1
        val_total += len(val_targets)
        
    #record--------------------------------------------------------------------------------------
    val_acces.append((val_correct/val_total).cpu().detach().data.numpy())
    val_acc_str = 'Accuracy: %f'%((val_correct/val_total).cpu().detach().data.numpy())
    print(val_acc_str) 
    val_losses.append((val_loss_sum/val_batches).cpu().detach().data.numpy())
    val_loss_str = 'Mean Validation Loss:: %f'%((val_loss_sum/val_batches).cpu().detach().data.numpy())
    print(val_loss_str)      
    acces.append((correct/total).cpu().detach().data.numpy())  
    acc_str = 'Accuracy: %f'%((correct/total).cpu().detach().data.numpy())
    print(acc_str)
    loss_str = 'Mean loss: %f'%((sum_loss/batches).cpu().detach().data.numpy())
    print(loss_str)
    losses.append((sum_loss/batches).cpu().detach().data.numpy())


Epoch [1/100], Step [512/0], Loss: 0.6931
Epoch [1/100], Step [1024/0], Loss: 0.6932
Epoch [1/100], Step [1536/0], Loss: 0.6931


KeyboardInterrupt: 